In [ ]:
from neuralop.models import FNO
from neuralop.training import AdamW
from neuralop.utils import count_model_params
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset, Subset
import torch
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from scipy.interpolate import interp1d
from os.path import dirname, join as pjoin
import os
import io
import scipy.io as sio
import urllib
from neuralop import Trainer
from neuralop.losses.data_losses import LpLoss
import glob

# データダウンロード

In [ ]:
def load_all_moments(root="../vlasov_random_data",max_cases=None):
    """
    root 以下の data_XXXX/moments.npz を全部読み込んで
    X: (Nsamples, 3, N), Y: (Nsamples, 1, N) を作る
    """
    X_list = []
    Y_list = []

    # data_0000, data_0001, ... を全部拾う
    folders = sorted(glob.glob(os.path.join(root, "data_*")))
    total = len(folders)

    if max_cases is not None:
        folders = folders[:max_cases]   # ← 指定数だけフォルダを使う

    print(f"Found {total} folders, loading {len(folders)} folders")


    for idx, folder in enumerate(folders):
        path = os.path.join(folder, "moments.npz")
        if not os.path.isfile(path):
            continue

        d = np.load(path)
        n = d["n"]
        u = d["u"]
        p = d["p"]
        dn_dx = d["dn_dx"]
        du_dx = d["du_dx"]
        dp_dx = d["dp_dx"]
        dq = d["dq_dx"]

        # すぐ torch tensor に変換
        X = torch.tensor(np.stack([n,u,p,dn_dx,du_dx,dp_dx],axis=1), dtype=torch.float32)
        Y = torch.tensor(dq[:,None,:], dtype=torch.float32)

        X_list.append(X)
        Y_list.append(Y)

        # プログレス表示
        if idx % 50 == 0:
            print(f"  loading... {idx}/{len(folders)}")

    # torch.cat で結合（高速 & メモリ節約）
    X = torch.cat(X_list, dim=0)
    Y = torch.cat(Y_list, dim=0)

    print("Done loading:", X.shape, Y.shape)
    return X, Y


# パラメータの設定

In [ ]:
# パラメータ
batch_size = 32 # バッチサイズ
num_epoch = 30 # エポック数
num_modes = 16 # フーリエ空間で使用するモードの数
num_channels = 64 # インプットとアウトプットの間の層の数
in_channels = 3 # インプット数
device = 'cuda'
root="../vlasov_random_data"
print("Using device:", device)

# データセット作成